In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [3]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [4]:
import emoji as emoji

In [5]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [6]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [7]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [8]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [9]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [10]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [11]:
embeddings = {}
with open('K:\Coding Blocks\Machine learning\glove6b50dtxt\glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [12]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [13]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

K:\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [15]:
from keras.layers import *
from keras.models import Sequential

In [16]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])


In [18]:
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import EarlyStopping 


checkpoint = ModelCheckpoint("best1_model.h5", monitor='val_loss', verbose=True, save_best_only=True)
earlystop = EarlyStopping(monitor='val_accuracy',patience=10)


In [19]:
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1,callbacks=[checkpoint,earlystop])

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 4s 38ms/step - loss: 1.5659 - acc: 0.3136 - val_loss: 1.6383 - val_acc: 0.2857

Epoch 00001: val_loss improved from inf to 1.63832, saving model to best1_model.h5
Epoch 2/40
118/118 [==============================] - 0s 1ms/step - loss: 1.5231 - acc: 0.3305 - val_loss: 1.6867 - val_acc: 0.2143

Epoch 00002: val_loss did not improve from 1.63832
Epoch 3/40
 32/118 [=======>......................] - ETA: 0s - loss: 1.5440 - acc: 0.2500

K:\Anaconda3\envs\py\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


118/118 [==============================] - 0s 1ms/step - loss: 1.4894 - acc: 0.3559 - val_loss: 1.6772 - val_acc: 0.2143

Epoch 00003: val_loss did not improve from 1.63832
Epoch 4/40
118/118 [==============================] - 0s 1ms/step - loss: 1.4462 - acc: 0.3814 - val_loss: 1.6255 - val_acc: 0.2857

Epoch 00004: val_loss improved from 1.63832 to 1.62549, saving model to best1_model.h5
Epoch 5/40
118/118 [==============================] - 0s 1ms/step - loss: 1.4062 - acc: 0.4492 - val_loss: 1.5457 - val_acc: 0.2857

Epoch 00005: val_loss improved from 1.62549 to 1.54575, saving model to best1_model.h5
Epoch 6/40
118/118 [==============================] - 0s 1ms/step - loss: 1.3307 - acc: 0.5000 - val_loss: 1.4732 - val_acc: 0.4286

Epoch 00006: val_loss improved from 1.54575 to 1.47324, saving model to best1_model.h5
Epoch 7/40
118/118 [==============================] - 0s 1ms/step - loss: 1.2499 - acc: 0.5169 - val_loss: 1.3988 - val_acc: 0.4286

Epoch 00007: val_loss improved fro

In [20]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 641us/step


[1.3664231300354004, 0.5892857313156128]

In [21]:
model.load_weights('best1_model.h5')

In [22]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 641us/step


[1.3825903960636683, 0.5714285969734192]

In [23]:
pred = model.predict_classes(emb_Xt)

In [24]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
😓
you brighten my day
😁
😓
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
